In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import volume_grating as vg
import volume_grating.utilities
from volume_grating.utilities.geometry import cartersian_to_spherical, spherical_to_cartesian, point_to_ndarray, vector_to_ndarray
from volume_grating.utilities.surface_fitting import *
from volume_grating.utilities.validation import validate_input_numeric
import matplotlib.pyplot as plt
from tqdm import tqdm

from volume_grating import *

from volume_grating.routines import sweep

import importlib

# GCS = vec.CoordSys3D('GCS')

In [ ]:
mat_lib = materials.cauchy_dict
pp_org = materials.Cauchy_2coeff(mat_lib['pp_original'])
air = materials.air

## Estimate dn from analytical reflection

$R=tanh^2(\frac{\pi \cdot \delta n \cdot d \cos\theta}{\lambda}) = 0.99$

In [ ]:
dn = np.arctanh(np.sqrt(0.99))*660e-9/np.pi/70e-6
print('dn =', dn)

# Define HOE

In [ ]:
thickness = 50e-6
dn = 0.01

In [ ]:
src1 = sources.Planewave(material=pp_org, direction=[0,0,-1], wavelength=660e-9)
src2 = sources.Planewave(material=pp_org, direction=[-1,0,1], wavelength=660e-9)
recording = illumination.Record(src1, src2)

In [ ]:
hoe = holograms.Hologram(thickness=thickness, material=pp_org, dn=dn, recording=recording)

### Define playback

In [ ]:
src3 = sources.Planewave(material=pp_org, direction=[0,0,-1], wavelength=660e-9)

In [ ]:
playback = illumination.Playback(src3)

# Define solver and engine

In [ ]:
wavelengths = np.linspace(650, 670, 101)*1e-9

In [ ]:
ka = engines.KogelnikAnalytical()
kt = engines.KogelnikTwoWave()

In [ ]:
solver = solvers.Response(hoe, playback=playback, engine=ka, order=1)

In [ ]:
DE, caches = solver.get_efficiency_at_points([GCS.origin], wavelengths=wavelengths, analytical_aid=True, tol=0.01, 
                                     max_nodes=int(1e6))
plt.plot(wavelengths/1e-9, DE[0], '.-')

In [ ]:
caches[0]["r_d"]

In [ ]:
plt.plot(wavelengths/1e-9, np.abs(caches[0]["r_d"])**2)

In [ ]:
solver = solvers.Response(hoe, playback=playback, engine=kt, order=1)

In [ ]:
param = solver.extract_params(wavelengths=662e-9)
param

In [ ]:
from scipy.integrate import solve_ivp

from volume_grating.engines import KogelnikTwoWave

In [ ]:
mesh_number = 2
analytical_aid = False

In [ ]:

mode = param['mode']
k_r = param['k_r']/1e6  # reference wave vector, now in 1/um unit
k_d = param['k_d']/1e6  # diffracted wave vector, now in 1/um unit
K = param['K']/1e6  # grating vector, now in 1/um unit
thickness = param['thickness']/1e-6  # now in um
n0 = param['n0']
dn = param['dn']
# wavelengths = param['wavelength']/1e-6  # now in um
wavelengths = np.array([661e-9])/1e-6

# pre-define ``matrix`` that will be updated later.
global matrix
matrix = np.ndarray(shape=(2, 2), dtype=np.complex)

# from approximation of (n0+dn)^2 ~ n0^2 + 2*n0*dn, so ep0 = n0^2, ep1~n0*dn
ep0 = n0 ** 2
ep1 = 2 * dn * n0

z = np.linspace(0.0, thickness, mesh_number)

efficiency = np.zeros_like(wavelengths)*np.nan

def _rhs(x, y):
    """
    A function to compute a state [dAr/dz, dAd/dz].

    :param x: indendent variable of the problem, i.e. z. But it is not needed for uniform gratings.
    :param y: current state, i.e. [Ar, Ad]
    :return [dAr/dz, dAd/dz]: (2,) ndarray
    """
    matrix_h = np.reshape(matrix, newshape=(2, 2))
    dy = np.matmul(matrix_h, y)
    return dy

if analytical_aid:
    eff_anlt, _, caches_anlt = KogelnikAnalytical.solve(param, **kwarg)
    s_anlt = caches_anlt["s"]

for i, wavelength in enumerate(wavelengths):
    k0 = 2 * np.pi / wavelength  # free space wavenumber
    beta = k0 * n0[i]

    gamma0 = k0**2 * ep0[i]
    gamma = k0**2 * ep1[i] / 2

    kz_r = k_r[i][-1]
    kz_d = k_d[i][-1]

    matrix = np.reshape(matrix, newshape=(2, 2))
    
    matrix[0, 0] = -(gamma0 - np.sum(k_r**2)) / 2 / 1j / kz_r
    
    matrix[0, 1] = -gamma/2/1j/kz_r
    matrix[1, 0] = -gamma/2/1j/kz_d

    matrix[1, 1] = -(gamma0 - np.sum(k_d[i]**2)) / 2 / 1j / kz_d
    
    matrix = np.reshape(matrix, newshape=(matrix.size,))

    y_guess = np.ndarray(shape=(2,), dtype=np.complex)

    if mode == "transmission":
        print("Transmission not doing anything yet.")

    if mode == "reflection":
        y_guess[0] = np.array([1.0], dtype=np.complex)
        y_guess[1] = np.array([.5], dtype=np.complex)
    
        result_detail = solve_ivp(_rhs,
                                 t_span=z,
                                 y0=y_guess)
        # diffracted wave solution
#         s_wave_sol = result_detail.y[1]
        

In [ ]:
np.reshape(matrix, (2,2))

In [ ]:
from scipy.optimize import minimize, least_squares, leastsq

In [ ]:
bv = np.zeros(shape=(2, 2), dtype=np.complex)*np.nan
bv[0, 0] = 1
bv[1, 1] = 0.0

def find_solution(rhs, t_span, y0, max_step=0.01):
    iv = np.array([1, y0], dtype=np.complex)
    result_detail = solve_ivp(rhs, t_span=t_span, y0=iv, max_step=max_step)
    solutions = result_detail.y
    return solutions, result_detail.t

def get_boundary_values(solutions):
    bv = np.zeros(shape=(solutions.shape[0], 2), dtype=np.complex)
    for i in range(solutions.shape[0]):
        bv[i][0] = solutions[i][0]
        bv[i][-1] = solutions[i][-1]
    return bv

def loss_func(x, rhs, t_span, bv):
        
    sol, _ = find_solution(rhs, t_span, x)
    bv_sol = get_boundary_values(sol)
    loss = np.nansum(np.abs(bv-bv_sol)**2)

    return loss

def solve_bvp_by_shooting(rhs, t_span, bv, y20 = 0.01):
    
    result = minimize(fun=loss_func, x0=y20, args=(rhs, t_span, bv))
#     result = leastsq(func=loss_func, x0=y20, args=(rhs, t_span, bv))
    
    return result

In [ ]:
z = np.linspace(0, thickness, 2)

sol, z_span = find_solution(_rhs, z, -0.2*1j)

plt.plot(z_span, np.abs(sol[0])**2)
plt.plot(z_span, np.abs(sol[1])**2)

plt.ylim(bottom=0)
plt.show()

In [ ]:
bvals = get_boundary_values(sol)
bvals

In [ ]:
result = solve_bvp_by_shooting(_rhs, z, bv, y20=0.5)

In [ ]:
loss = loss_func(result.x, _rhs, z, bv)
loss

In [ ]:
z = np.linspace(0, thickness, 2)

sol, z_span = find_solution(_rhs, z, result.x)

plt.plot(z_span, np.abs(sol[0])**2)
plt.plot(z_span, np.abs(sol[1])**2)

plt.ylim(bottom=0)
plt.show()

In [ ]:
break

In [ ]:
# DE, _ = solver.get_efficiency_at_points([GCS.origin], wavelengths=wavelengths, analytical_aid=True, tol=0.001, 
#                                      max_nodes=int(1e6))

# plt.plot(wavelengths/1e-9, DE[0], '.-')

In [ ]:
def get_bandwidth_spectral_at_points(points:Callable):
    pass

In [ ]:
kt.solve_default

In [ ]:
break

In [ ]:
K_mag = solver.get_k_hologram_at_points([GCS.origin])[0].magnitude()
bw_spectral = np.pi*(4*np.pi*pp_org.n(660e-9))/hoe.thickness/K_mag**2 * 2 * 2

bw_spectral

In [ ]:
def update_angle(angle, solver):
    src = solver.playback.source
    
    src.direction = [-np.sin(np.deg2rad(angle)), 0, -np.cos(np.deg2rad(angle))]
    
    return solver

In [ ]:
angles = np.linspace(-5, 5, 201)

solver_funcs = [solvers.Response.get_efficiency_at_points]

DE = sweep(angles, [GCS.origin], update_func=update_angle, solver=solver, solver_funcs=solver_funcs, verbose=True)

In [ ]:
plt.plot(angles, -DE.squeeze())